Rabbit Question

In [8]:
class RabbitState:
    def __init__(self, path):
        self.path = path

    def goalTest(self):
        return self.path == ["w1", "w2", "w3", "-", "-", "-", "e1", "e2", "e3"]

    def moveGen(self):
        children = []
        for i in range(9):
            current = self.path[i]

            if current.startswith("e"):
                if i+1 < 9 and self.path[i+1] == "-":
                    new_path = self.path.copy()
                    new_path[i], new_path[i+1] = "-", current
                    children.append(RabbitState(new_path))

                if i+2 < 9 and self.path[i+1] != "-" and self.path[i+2] == "-":
                    new_path = self.path.copy()
                    new_path[i], new_path[i+2] = "-", current
                    children.append(RabbitState(new_path))

            elif current.startswith("w"):
                if i-1 >= 0 and self.path[i-1] == "-":
                    new_path = self.path.copy()
                    new_path[i], new_path[i-1] = "-", current
                    children.append(RabbitState(new_path))

                if i-2 >= 0 and self.path[i-1] != "-" and self.path[i-2] == "-":
                    new_path = self.path.copy()
                    new_path[i], new_path[i-2] = "-", current
                    children.append(RabbitState(new_path))

        return children

    def __eq__(self, other):
        return self.path == other.path

    def __hash__(self):
        return hash(tuple(self.path))

    def __str__(self):
        return " ".join(self.path)


Bridge Crossing State Code

In [10]:
class BridgeState:
    def __init__(self, left, right, umbrella, time_left):
        self.left = left
        self.right = right
        self.umbrella = umbrella
        self.time_left = time_left

    def goalTest(self):
        return len(self.left) == 0 and len(self.right) == 4 and self.time_left >= 0

    def moveGen(self):
        time_taken = {
            "Amogh": 5,
            "Ameya": 10,
            "Grandma": 20,
            "Grandpa": 25
        }

        children = []

        if self.umbrella == "left":
            for i in range(len(self.left)):
                for j in range(i, len(self.left)):
                    p1 = self.left[i]
                    p2 = self.left[j]
                    t = max(time_taken[p1], time_taken[p2])
                    if self.time_left - t >= 0:
                        new_left = self.left.copy()
                        new_right = self.right.copy()
                        new_left.remove(p1)
                        if p1 != p2:
                            new_left.remove(p2)
                            new_right.append(p2)
                        new_right.append(p1)
                        children.append(BridgeState(new_left, new_right, "right", self.time_left - t))

        elif self.umbrella == "right":
            for person in self.right:
                t = time_taken[person]
                if self.time_left - t >= 0:
                    new_left = self.left.copy()
                    new_right = self.right.copy()
                    new_right.remove(person)
                    new_left.append(person)
                    children.append(BridgeState(new_left, new_right, "left", self.time_left - t))

        return children

    def __eq__(self, other):
        return self.left == other.left and self.right == other.right and self.umbrella == other.umbrella and self.time_left == other.time_left

    def __hash__(self):
        return hash((tuple(self.left), tuple(self.right), self.umbrella, self.time_left))

    def __str__(self):
        return f"Left: {self.left}, Right: {self.right}, Umbrella: {self.umbrella}, Time left: {self.time_left}"


BFS and DFS paths

In [12]:
def removeSeen(children, OPEN, CLOSED):
    open_nodes = [node for node, parent in OPEN]
    closed_nodes = [node for node, parent in CLOSED]
    new_nodes = [c for c in children if c not in open_nodes and c not in closed_nodes]
    return new_nodes

def reconstructPath(node_pair, CLOSED):
    path = []
    parent_map = {}
    for node, parent in CLOSED:
        parent_map[node] = parent 
    node, parent = node_pair
    path.append(node)
    while parent is not None:
        path.append(parent)
        parent = parent_map[parent]
    return path

def bfs(start):
    OPEN = [(start, None)]
    CLOSED  = []
    while OPEN:
        node_pair = OPEN.pop(0)
        N, parent = node_pair

        if N.goalTest():
            print("Goal found")
            path = reconstructPath(node_pair, CLOSED)
            path.reverse()
            for p in path:
                print("->", p)
            return 
        else:
            CLOSED.append(node_pair)
            children = N.moveGen()
            new_nodes = removeSeen(children, OPEN, CLOSED)
            new_pairs = [(c, N) for c in new_nodes]
            OPEN = OPEN + new_pairs
    return []

def dfs(start):
    OPEN = [(start, None)]
    CLOSED  = []
    while OPEN:
        node_pair = OPEN.pop(0)
        N, parent = node_pair

        if N.goalTest():
            print("Goal found")
            path = reconstructPath(node_pair, CLOSED)
            path.reverse()
            for p in path:
                print("->", p)
            return 
        else:
            CLOSED.append(node_pair)
            children = N.moveGen()
            new_nodes = removeSeen(children, OPEN, CLOSED)
            new_pairs = [(c, N) for c in new_nodes]
            OPEN = new_pairs + OPEN
    return []


Rabbit problem

In [14]:
print("=== RABBIT PROBLEM ===")
initial = ["-", "e1", "e2", "e3", "-", "w1", "w2", "w3", "-"]
start_state = RabbitState(initial)
print("BFS:")
bfs(start_state)
print("\nDFS:")
dfs(start_state)



=== RABBIT PROBLEM ===
BFS:
Goal found
-> - e1 e2 e3 - w1 w2 w3 -
-> - e1 e2 - e3 w1 w2 w3 -
-> - e1 e2 w1 e3 - w2 w3 -
-> - e1 e2 w1 e3 w2 - w3 -
-> - e1 e2 w1 - w2 e3 w3 -
-> - e1 - w1 e2 w2 e3 w3 -
-> - - e1 w1 e2 w2 e3 w3 -
-> - w1 e1 - e2 w2 e3 w3 -
-> w1 - e1 - e2 w2 e3 w3 -
-> w1 - - e1 e2 w2 e3 w3 -
-> w1 - - e1 e2 w2 - w3 e3
-> w1 - - e1 - w2 e2 w3 e3
-> w1 - - - e1 w2 e2 w3 e3
-> w1 - - w2 e1 - e2 w3 e3
-> w1 - - w2 e1 w3 e2 - e3
-> w1 - - w2 e1 w3 - e2 e3
-> w1 - - w2 - w3 e1 e2 e3
-> w1 - - w2 w3 - e1 e2 e3
-> w1 - w3 w2 - - e1 e2 e3
-> w1 w2 w3 - - - e1 e2 e3

DFS:
Goal found
-> - e1 e2 e3 - w1 w2 w3 -
-> - e1 e2 - e3 w1 w2 w3 -
-> - e1 e2 w1 e3 - w2 w3 -
-> - e1 e2 w1 e3 w2 - w3 -
-> - e1 e2 w1 - w2 e3 w3 -
-> - e1 - w1 e2 w2 e3 w3 -
-> - - e1 w1 e2 w2 e3 w3 -
-> - w1 e1 - e2 w2 e3 w3 -
-> w1 - e1 - e2 w2 e3 w3 -
-> w1 - - e1 e2 w2 e3 w3 -
-> w1 - - e1 e2 w2 - w3 e3
-> w1 - - e1 - w2 e2 w3 e3
-> w1 - - - e1 w2 e2 w3 e3
-> w1 - - w2 e1 - e2 w3 e3
-> w1 - w2 - e1 - e2 w3 e3

Bridge Problem

In [16]:
print("\n=== BRIDGE PROBLEM ===")
start_state = BridgeState(["Amogh", "Ameya", "Grandma", "Grandpa"], [], "left", 60)
print("BFS:")
bfs(start_state)
print("\nDFS:")
dfs(start_state)



=== BRIDGE PROBLEM ===
BFS:
Goal found
-> Left: ['Amogh', 'Ameya', 'Grandma', 'Grandpa'], Right: [], Umbrella: left, Time left: 60
-> Left: ['Grandma', 'Grandpa'], Right: ['Ameya', 'Amogh'], Umbrella: right, Time left: 50
-> Left: ['Grandma', 'Grandpa', 'Ameya'], Right: ['Amogh'], Umbrella: left, Time left: 40
-> Left: ['Ameya'], Right: ['Amogh', 'Grandpa', 'Grandma'], Umbrella: right, Time left: 15
-> Left: ['Ameya', 'Amogh'], Right: ['Grandpa', 'Grandma'], Umbrella: left, Time left: 10
-> Left: [], Right: ['Grandpa', 'Grandma', 'Amogh', 'Ameya'], Umbrella: right, Time left: 0

DFS:
Goal found
-> Left: ['Amogh', 'Ameya', 'Grandma', 'Grandpa'], Right: [], Umbrella: left, Time left: 60
-> Left: ['Grandma', 'Grandpa'], Right: ['Ameya', 'Amogh'], Umbrella: right, Time left: 50
-> Left: ['Grandma', 'Grandpa', 'Ameya'], Right: ['Amogh'], Umbrella: left, Time left: 40
-> Left: ['Ameya'], Right: ['Amogh', 'Grandpa', 'Grandma'], Umbrella: right, Time left: 15
-> Left: ['Ameya', 'Amogh'], Righ